In [3]:
from astropy.io import fits
import numpy as np


paths = [f"data/ETC/F430M/image/", f"data/ETC/F480M/image/"]

file_det = "image_detector.fits"
file_ngroups = "image_ngroups_map.fits"
file_sat = "image_saturation.fits"
file_snr = "image_snr.fits"

## Loading files in for each filter

In [4]:
# F430M
ngroups_430 = fits.open(paths[0] + file_ngroups)[0]
det_430 = fits.open(paths[0] + file_det)[0]

psf_430 = np.array(det_430.data)
total_flux_430 = np.nansum(psf_430)

# F480M
ngroups_480 = fits.open(paths[1] + file_ngroups)[0]
det_480 = fits.open(paths[1] + file_det)[0]

psf_480 = np.array(det_480.data)
total_flux_480 = np.nansum(psf_480)

In [6]:
n = 10
groups = np.arange(1, n+1)
losts_430, losts_480 = [], []

for i in groups:
    # saturation mask
    mask_430 = np.where(ngroups_430.data <= i, 1, 0)
    mask_480 = np.where(ngroups_480.data <= i, 1, 0)

    # lost flux
    lost_430 = np.nansum(psf_430 * mask_430)
    lost_480 = np.nansum(psf_480 * mask_480)

    # appending
    losts_430.append(lost_430 / total_flux_430)
    losts_480.append(lost_480 / total_flux_480)


losts_430 = np.array(losts_430)
losts_480 = np.array(losts_480)